In [1]:
from fastai.text.all import *

In [2]:
awd =  AWD_LSTM(vocab_sz=3,emb_sz=5,n_hid=6,n_layers=2)

In [3]:
x = torch.randint(3,(1,4))
x

tensor([[2, 1, 1, 2]])

In [4]:
awd(x)

tensor([[[-0.1139, -0.0267,  0.0203, -0.0664,  0.0800],
         [-0.1726, -0.0404,  0.0382, -0.0478,  0.1060],
         [-0.2043, -0.0484,  0.0459, -0.0246,  0.1102],
         [-0.2218, -0.0539,  0.0470, -0.0081,  0.1079]]],
       grad_fn=<TransposeBackward0>)

In [31]:
lstm = nn.LSTM(5,5)

In [32]:
x = torch.randint(3, (1,1,5), dtype=torch.float)
x

tensor([[[2., 2., 1., 1., 0.]]])

In [33]:
out_1 = lstm(x)

In [34]:
out_1

(tensor([[[-0.0267,  0.0024, -0.0513,  0.0225,  0.0776]]],
        grad_fn=<StackBackward>),
 (tensor([[[-0.0267,  0.0024, -0.0513,  0.0225,  0.0776]]],
         grad_fn=<StackBackward>),
  tensor([[[-0.1169,  0.0155, -0.3378,  0.0495,  0.2995]]],
         grad_fn=<StackBackward>)))

In [39]:
lstm(out_1[0])

(tensor([[[ 0.0753, -0.0504, -0.1746, -0.0545, -0.0190]]],
        grad_fn=<StackBackward>),
 (tensor([[[ 0.0753, -0.0504, -0.1746, -0.0545, -0.0190]]],
         grad_fn=<StackBackward>),
  tensor([[[ 0.2047, -0.0926, -0.3339, -0.0869, -0.0517]]],
         grad_fn=<StackBackward>)))

### Testing the Module

In [40]:
#exports
@delegates()
class ActivationStats(HookCallback):
    "Callback that record the mean and std of activations."
    run_before=TrainEvalCallback
    def __init__(self, with_hist=False, **kwargs):
        super().__init__(**kwargs)
        self.with_hist = with_hist

    def before_fit(self):
        "Initialize stats."
        super().before_fit()
        self.stats = L()

    def hook(self, m, i, o):
        if isinstance(o, tuple): return self.hook_multi_ouput(o)
        o = o.float()
        res = {'mean': o.mean().item(), 'std': o.std().item(),
               'near_zero': (o<=0.05).long().sum().item()/o.numel()}
        if self.with_hist: res['hist'] = o.histc(40,0,10)
        return res
    
    def hook_multi_ouput(self,o_tuple):
        "For outputs of RNN which are [nested] tuples of tensors"
        res = []
        for o in self._flatten_tuple(o_tuple):
            if not(isinstance(o, Tensor)): continue
            res.append(self.hook(None, None, o))
        return res

    def _flatten_tuple(self, o_tuple):
        "Recursively flatten a [nested] tuple"
        res = []
        for it in o_tuple:
            if isinstance(it, tuple): res += self._flatten_tuple(it)
            else: res += [it]
        return tuple(res)

    def after_batch(self):
        "Take the stored results and puts it in `self.stats`"
        if self.training and (self.every is None or self.train_iter%self.every == 0):
            self.stats.append(self.hooks.stored)
        super().after_batch()

    def layer_stats(self, idx):
        lstats = self.stats.itemgot(idx)
        return L(lstats.itemgot(o) for o in ('mean','std','near_zero'))

    def hist(self, idx):
        res = self.stats.itemgot(idx).itemgot('hist')
        return torch.stack(tuple(res)).t().float().log1p()

    def color_dim(self, idx, figsize=(10,5), ax=None):
        "The 'colorful dimension' plot"
        res = self.hist(idx)
        if ax is None: ax = subplots(figsize=figsize)[1][0]
        ax.imshow(res, origin='lower')
        ax.axis('off')

    def plot_layer_stats(self, idx):
        _,axs = subplots(1, 3, figsize=(12,3))
        for o,ax,title in zip(self.layer_stats(idx),axs,('mean','std','% near zero')):
            ax.plot(o)
            ax.set_title(title)

In [43]:
from fastai.test_utils import synth_learner, synth_dbunch

In [49]:
def synth_dbunch(a=2, b=3, bs=16, n_train=10, n_valid=2, cuda=False):
    def get_data(n):
        x = torch.randn(bs*n, 1)
        return TensorDataset(x, a*x + b + 0.1*torch.randn(bs*n, 1))
    train_ds = get_data(n_train)
    valid_ds = get_data(n_valid)
    device = default_device() if cuda else None
    train_dl = TfmdDL(train_ds, bs=bs, shuffle=True, num_workers=0)
    valid_dl = TfmdDL(valid_ds, bs=bs, num_workers=0)
    return DataLoaders(train_dl, valid_dl, device=device)

In [47]:
learn = synth_learner(model=nn.LSTM(1,1))

In [48]:
learn.fit(3)

RuntimeError: input must have 3 dimensions, got 2

In [ ]:
l